In [10]:

import datetime
import os
import pandas as pd
from typing import List
from scraper.Comment import Comment
from scraper.JsonLoader import JsonLoader
from scraper.RedditJsonLoader import RedditJsonLoader
from scraper.RawCommentParser import RawCommentParser
from scraper.RedditCommentLoader import RedditCommentLoader
from scraper.RedditCommentSerializer import RedditCommentSerializer

In [11]:
def print_comments(comments, level=0):
    for comment in comments:
        print('  ' * level + comment.body)
        print_comments(comment.children, level + 1)

In [12]:
def validate_mode_and_path(mode, local_data_path):
    """
    Validate the mode and local data path to ensure they are compatible.
    
    Args:
        mode (str): The mode to use ('RAW_JSON', 'JSON', 'URL', 'CSV')
        local_data_path (str): Path to the local data file
        
    Returns:
        bool: True if valid, False otherwise
    """
    # Define supported modes and file extensions
    supported_modes = ['RAW_JSON', 'JSON', 'URL', 'CSV']
    file_extensions = {
        'json': ['JSON', 'RAW_JSON'],
        'csv': ['CSV']
    }

    # Validate the mode
    if mode not in supported_modes:
        print(f'Invalid mode: Select one of {", ".join(supported_modes)}')
        return False

    # For non-URL modes, validate that the file exists and has matching extension
    if mode != 'URL':
        if not os.path.exists(local_data_path):
            print('Local data file not found. LOCAL_DATA_PATH must be set for any mode other than URL')
            return False
        
        # Check if file extension matches the mode
        file_ext = os.path.splitext(local_data_path)[1].lower().replace('.', '')
        valid_modes = file_extensions.get(file_ext, [])
        
        if mode not in valid_modes:
            print(f'Mode {mode} is not compatible with file extension .{file_ext}')
            print(f'For .{file_ext} files, use one of: {", ".join(valid_modes)}')
            return False
            
    return True

In [13]:
# Get mode and local data path
mode = os.getenv('MODE', 'URL')
output_path = os.getenv('BASE_PATH', 'Dataset/output')
local_data_path = None
if mode == 'URL':
    local_data_path = None  # Not needed for URL mode
elif mode == 'CSV':
    local_data_path = os.getenv('LOCAL_DATA_PATH', 'Dataset/sample/comments.csv')
else:  # JSON or RAW_JSON modes
    local_data_path = os.getenv('LOCAL_DATA_PATH', 'Dataset/sample/reddit.json')

if not validate_mode_and_path(mode, local_data_path):
    exit(1)

data = None
comments = None
if mode == 'RAW_JSON':
    json_loader = JsonLoader()
    data = json_loader.load_raw(local_data_path)
    comments: List[Comment] = RawCommentParser().parse_comments(data)
elif mode == 'JSON':
    json_loader = RedditJsonLoader()
    comments: List[Comment] = json_loader.load_comments(local_data_path)
elif mode == 'URL':
    example_url =\
        'https://www.reddit.com/r/diabetes_t1/comments/1h9k636/type_1s_who_have_taken_ozempic_what_was_your/'
    data = RedditCommentLoader().get_comments(example_url)
    comments: List[Comment] = RawCommentParser().parse_comments(data)
elif mode == 'CSV':
    print('Use "df = pd.read_csv(csv_filename)" to load the CSV file into a DataFrame')
    df = pd.read_csv(local_data_path)
else:
    print('Invalid mode')
    exit(1)

print_comments(comments[:1])

T1 for close to 40 years. I’m on a Tandem TSlim pump with a Dexcom G6. I have been on Mounjaro for close to 18 months. My daily insulin usage almost immediately went from 110-120 units a day to 40-50. My A1C went from a 7.6 to a 5.6. It was a 6.0 4 months after starting Mounjaro. I’ve also lost 85-90 pounds. These drugs are a game changer for T1 management. I haven’t had any serious side effects and I’ve never increased my dosage past 7.5.
  Very similar results for me, too
  My experience is almost identical to this as well. The only thing that has ever improved my control/health as much as GLP-1 drugs is when I switched to a CGM that integrated with my pump (also on tandem + dexcom combo).
  Pretty much my results except my A1c is lower and I only lost sixty pounds. I think everyone should take mounjaro if possible. It’s a miracle drug.
  T1 40+ years. Went from TTD around 45-55 units a day to 25-40. Lost 45 pounds. Ozempic also reduces inflammation so it takes away that round face t

In [20]:
serializer = RedditCommentSerializer()
df = serializer.to_dataframe(comments)

print(f'Dataframe shape: {df.shape}')
print(f'Columns: {df.columns}')
df.info()

Dataframe shape: (107, 11)
Columns: Index(['id', 'author', 'body', 'created_utc', 'score', 'parent_id', 'depth',
       'ups', 'downs', 'num_reports', 'report_reasons'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 0 to 106
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              107 non-null    object 
 1   author          107 non-null    object 
 2   body            107 non-null    object 
 3   created_utc     107 non-null    float64
 4   score           107 non-null    int64  
 5   parent_id       107 non-null    object 
 6   depth           107 non-null    int64  
 7   ups             107 non-null    int64  
 8   downs           107 non-null    int64  
 9   num_reports     0 non-null      object 
 10  report_reasons  0 non-null      object 
dtypes: float64(1), int64(4), object(6)
memory usage: 9.3+ KB


In [15]:
""" Uncomment to test the export functionality
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H_%M_%S")
if not os.path.exists(output_path):
    os.makedirs(output_path)
"""

' Uncomment to test the export functionality\ntimestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H_%M_%S")\nif not os.path.exists(output_path):\n    os.makedirs(output_path)\n'

In [16]:
""" Uncomment to save the comments to a CSV file
csv_filename = os.path.join(output_path, f'{timestamp}_comments.csv')
serializer.write_to_csv(comments, csv_filename)

print(f"Comments saved to {csv_filename}")
"""

' Uncomment to save the comments to a CSV file\ncsv_filename = os.path.join(output_path, f\'{timestamp}_comments.csv\')\nserializer.write_to_csv(comments, csv_filename)\n\nprint(f"Comments saved to {csv_filename}")\n'

In [17]:
""" Uncomment to save as JSON
json_filename = os.path.join(output_path, f'{timestamp}_comments.json')
serializer.write_to_json(comments, json_filename)

print(f"Comments saved to {json_filename}")
"""

' Uncomment to save as JSON\njson_filename = os.path.join(output_path, f\'{timestamp}_comments.json\')\nserializer.write_to_json(comments, json_filename)\n\nprint(f"Comments saved to {json_filename}")\n'

In [18]:
""" Uncomment to test loading the CSV file
df = pd.read_csv(csv_filename)
df.head()
"""

' Uncomment to test loading the CSV file\ndf = pd.read_csv(csv_filename)\ndf.head()\n'